# Previsão de Tendência da PETR4.SA usando Machine Learning

## Objetivo
Desenvolver um modelo de machine learning para prever a tendência (↑ ou ↓) de uma série temporal financeira da PETR4.SA, com acurácia mínima de 75% no conjunto de teste (últimos 30 dias).

## Outline do Projeto
1. **Importação de Bibliotecas**
2. **Carregamento e Exploração dos Dados**
3. **Pré-processamento dos Dados**
4. **Engenharia de Features**
5. **Divisão Treino/Teste**
6. **Treinamento de Múltiplos Modelos**
7. **Avaliação dos Modelos**
8. **Previsões e Visualização dos Resultados**

---

## 1. Importação de Bibliotecas

In [ ]:
# Bibliotecas para manipulação de dados
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Bibliotecas para visualização
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Bibliotecas para download de dados financeiros
import yfinance as yf

# Bibliotecas para machine learning
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.pipeline import Pipeline

# Bibliotecas para análise técnica
import talib

# Configurações
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Ignorar warnings
import warnings
warnings.filterwarnings('ignore')

print("✅ Bibliotecas importadas com sucesso!")

## 2. Carregamento e Exploração dos Dados

Vamos carregar os dados históricos da PETR4.SA usando a biblioteca yfinance e explorar a estrutura dos dados.

In [ ]:
# Definir o período de análise (2 anos de dados históricos)
end_date = datetime.now()
start_date = end_date - timedelta(days=730)  # ~2 anos

# Carregar dados da PETR4.SA
ticker = "PETR4.SA"
print(f"Carregando dados de {ticker} de {start_date.strftime('%Y-%m-%d')} até {end_date.strftime('%Y-%m-%d')}")

data = yf.download(ticker, start=start_date, end=end_date, progress=False)

print(f"\n📊 Dados carregados: {len(data)} registros")
print(f"Período: {data.index[0].strftime('%Y-%m-%d')} a {data.index[-1].strftime('%Y-%m-%d')}")

# Visualizar primeiras e últimas linhas
print("\n🔍 Primeiras 5 linhas:")
display(data.head())

print("\n🔍 Últimas 5 linhas:")
display(data.tail())

# Informações básicas sobre os dados
print("\n📈 Informações dos dados:")
print(data.info())

print("\n📊 Estatísticas descritivas:")
display(data.describe())

In [ ]:
# Verificar valores ausentes
print("🔍 Verificando valores ausentes:")
missing_values = data.isnull().sum()
print(missing_values)

if missing_values.sum() > 0:
    print("\n⚠️ Encontrados valores ausentes. Será necessário tratamento.")
else:
    print("\n✅ Nenhum valor ausente encontrado!")

# Visualização inicial dos preços
fig = make_subplots(rows=2, cols=2, 
                    subplot_titles=('Preço de Fechamento', 'Volume', 'High-Low', 'Open-Close'),
                    vertical_spacing=0.08)

# Preço de fechamento
fig.add_trace(go.Scatter(x=data.index, y=data['Close'], name='Close', line=dict(color='blue')), row=1, col=1)

# Volume
fig.add_trace(go.Scatter(x=data.index, y=data['Volume'], name='Volume', line=dict(color='orange')), row=1, col=2)

# High-Low
fig.add_trace(go.Scatter(x=data.index, y=data['High'] - data['Low'], name='High-Low', line=dict(color='green')), row=2, col=1)

# Open-Close
fig.add_trace(go.Scatter(x=data.index, y=data['Close'] - data['Open'], name='Close-Open', line=dict(color='red')), row=2, col=2)

fig.update_layout(height=600, title_text="📈 Análise Exploratória - PETR4.SA", showlegend=False)
fig.show()

## 3. Pré-processamento dos Dados

Nesta seção, vamos limpar e preparar os dados para análise.

In [ ]:
# Criar uma cópia dos dados para processamento
df = data.copy()

# Renomear colunas para facilitar o uso
df.columns = ['open', 'high', 'low', 'close', 'adj_close', 'volume']

# Remover dados ausentes (se houver)
df = df.dropna()

# Criar variável target: 1 se o preço subiu, 0 se desceu
df['price_change'] = df['close'].pct_change()
df['target'] = (df['price_change'] > 0).astype(int)

# Remover a primeira linha (NaN devido ao pct_change)
df = df.dropna()

print(f"📊 Dados após pré-processamento: {len(df)} registros")
print(f"\n🎯 Distribuição da variável target:")
target_dist = df['target'].value_counts()
print(f"Descida (0): {target_dist[0]} ({target_dist[0]/len(df)*100:.1f}%)")
print(f"Subida (1): {target_dist[1]} ({target_dist[1]/len(df)*100:.1f}%)")

# Visualizar distribuição do target
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Gráfico de barras
target_dist.plot(kind='bar', ax=ax1, color=['red', 'green'])
ax1.set_title('Distribuição da Tendência')
ax1.set_xlabel('Tendência (0=Descida, 1=Subida)')
ax1.set_ylabel('Frequência')
ax1.tick_params(axis='x', rotation=0)

# Série temporal da tendência
ax2.plot(df.index, df['target'], alpha=0.7, color='purple')
ax2.set_title('Tendência ao Longo do Tempo')
ax2.set_xlabel('Data')
ax2.set_ylabel('Tendência (0=Descida, 1=Subida)')

plt.tight_layout()
plt.show()

display(df.head())

## 4. Engenharia de Features

Vamos criar indicadores técnicos e features que podem ajudar na previsão da tendência.

In [ ]:
def create_technical_features(df):
    """
    Cria features técnicas para análise de séries temporais financeiras
    """
    df_features = df.copy()
    
    # ========== FEATURES BÁSICAS ==========
    # Retornos percentuais
    for period in [1, 2, 3, 5, 10]:
        df_features[f'return_{period}d'] = df_features['close'].pct_change(period)
    
    # Volatilidade (desvio padrão dos retornos)
    for period in [5, 10, 20]:
        df_features[f'volatility_{period}d'] = df_features['price_change'].rolling(period).std()
    
    # ========== MÉDIAS MÓVEIS ==========
    for period in [5, 10, 20, 50]:
        df_features[f'sma_{period}'] = df_features['close'].rolling(period).mean()
        df_features[f'price_sma_{period}_ratio'] = df_features['close'] / df_features[f'sma_{period}']
    
    # Médias móveis exponenciais
    for period in [12, 26]:
        df_features[f'ema_{period}'] = df_features['close'].ewm(span=period).mean()
    
    # ========== INDICADORES TÉCNICOS COM TALIB ==========
    try:
        # RSI (Relative Strength Index)
        df_features['rsi_14'] = talib.RSI(df_features['close'].values, timeperiod=14)
        
        # MACD
        macd, macd_signal, macd_hist = talib.MACD(df_features['close'].values)
        df_features['macd'] = macd
        df_features['macd_signal'] = macd_signal
        df_features['macd_histogram'] = macd_hist
        
        # Bollinger Bands
        bb_upper, bb_middle, bb_lower = talib.BBANDS(df_features['close'].values)
        df_features['bb_upper'] = bb_upper
        df_features['bb_lower'] = bb_lower
        df_features['bb_position'] = (df_features['close'] - bb_lower) / (bb_upper - bb_lower)
        
        # Stochastic Oscillator
        stoch_k, stoch_d = talib.STOCH(df_features['high'].values, df_features['low'].values, df_features['close'].values)
        df_features['stoch_k'] = stoch_k
        df_features['stoch_d'] = stoch_d
        
        # Williams %R
        df_features['williams_r'] = talib.WILLR(df_features['high'].values, df_features['low'].values, df_features['close'].values)
        
        print("✅ Indicadores técnicos (TA-Lib) criados com sucesso!")
    except:
        print("⚠️ TA-Lib não disponível. Criando indicadores manualmente...")
        
        # RSI manual
        delta = df_features['close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        rs = gain / loss
        df_features['rsi_14'] = 100 - (100 / (1 + rs))
        
        # MACD manual
        ema_12 = df_features['close'].ewm(span=12).mean()
        ema_26 = df_features['close'].ewm(span=26).mean()
        df_features['macd'] = ema_12 - ema_26
        df_features['macd_signal'] = df_features['macd'].ewm(span=9).mean()
        df_features['macd_histogram'] = df_features['macd'] - df_features['macd_signal']
    
    # ========== FEATURES DE VOLUME ==========
    df_features['volume_sma_10'] = df_features['volume'].rolling(10).mean()
    df_features['volume_ratio'] = df_features['volume'] / df_features['volume_sma_10']
    
    # ========== FEATURES DE PADRÕES DE PREÇO ==========
    # High-Low range
    df_features['hl_range'] = (df_features['high'] - df_features['low']) / df_features['close']
    
    # Open-Close range
    df_features['oc_range'] = (df_features['close'] - df_features['open']) / df_features['open']
    
    # Posição do fechamento no range do dia
    df_features['close_position'] = (df_features['close'] - df_features['low']) / (df_features['high'] - df_features['low'])
    
    # ========== FEATURES TEMPORAIS ==========
    df_features['day_of_week'] = df_features.index.dayofweek
    df_features['month'] = df_features.index.month
    df_features['quarter'] = df_features.index.quarter
    
    return df_features

# Criar features
print("🔨 Criando features técnicas...")
df_with_features = create_technical_features(df)

print(f"✅ Features criadas! Total de colunas: {len(df_with_features.columns)}")
print(f"📊 Shape dos dados: {df_with_features.shape}")

# Remover linhas com NaN (criados pelos indicadores)
df_clean = df_with_features.dropna()
print(f"📊 Dados limpos: {len(df_clean)} registros")

# Mostrar algumas features criadas
feature_columns = [col for col in df_clean.columns if col not in ['open', 'high', 'low', 'close', 'adj_close', 'volume', 'price_change', 'target']]
print(f"\n🎯 Features criadas ({len(feature_columns)}):")
for i, feat in enumerate(feature_columns[:10]):
    print(f"{i+1:2d}. {feat}")
if len(feature_columns) > 10:
    print(f"    ... e mais {len(feature_columns) - 10} features")

In [ ]:
# Analisar correlação das features com o target
correlations = df_clean[feature_columns + ['target']].corr()['target'].abs().sort_values(ascending=False)

print("🎯 Top 15 features mais correlacionadas com o target:")
print(correlations.head(16)[1:])  # Excluir o próprio target

# Visualizar correlações
plt.figure(figsize=(12, 8))
top_features = correlations.head(16)[1:15]  # Top 15 excluindo target
plt.barh(range(len(top_features)), top_features.values)
plt.yticks(range(len(top_features)), top_features.index)
plt.xlabel('Correlação Absoluta com Target')
plt.title('Top 15 Features por Correlação com Target')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

# Selecionar features para o modelo
selected_features = correlations.head(21)[1:].index.tolist()  # Top 20 features
print(f"\n✅ Selecionadas {len(selected_features)} features para o modelo")

## 5. Divisão Treino/Teste

Vamos dividir os dados considerando que os últimos 30 dias serão nosso conjunto de teste.

In [ ]:
# Preparar dados para machine learning
X = df_clean[selected_features]
y = df_clean['target']

print(f"📊 Preparação dos dados:")
print(f"Features (X): {X.shape}")
print(f"Target (y): {y.shape}")

# Divisão temporal: últimos 30 dias para teste
split_date = df_clean.index[-30]  # 30 dias antes do final
print(f"\n📅 Data de divisão: {split_date.strftime('%Y-%m-%d')}")

# Conjuntos de treino e teste
X_train = X[X.index < split_date]
X_test = X[X.index >= split_date]
y_train = y[y.index < split_date]
y_test = y[y.index >= split_date]

print(f"\n📊 Divisão dos dados:")
print(f"Treino: {len(X_train)} registros ({X_train.index[0].strftime('%Y-%m-%d')} a {X_train.index[-1].strftime('%Y-%m-%d')})")
print(f"Teste:  {len(X_test)} registros ({X_test.index[0].strftime('%Y-%m-%d')} a {X_test.index[-1].strftime('%Y-%m-%d')})")

# Verificar distribuição do target em cada conjunto
print(f"\n🎯 Distribuição do target:")
print(f"Treino - Descida: {(y_train==0).sum()} ({(y_train==0).mean()*100:.1f}%), Subida: {(y_train==1).sum()} ({(y_train==1).mean()*100:.1f}%)")
print(f"Teste  - Descida: {(y_test==0).sum()} ({(y_test==0).mean()*100:.1f}%), Subida: {(y_test==1).sum()} ({(y_test==1).mean()*100:.1f}%)")

# Visualizar a divisão
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))

# Preços com divisão
ax1.plot(df_clean.index, df_clean['close'], label='Preço de Fechamento', alpha=0.8)
ax1.axvline(x=split_date, color='red', linestyle='--', alpha=0.8, label=f'Divisão Treino/Teste')
ax1.fill_between(X_train.index, df_clean.loc[X_train.index, 'close'].min(), 
                df_clean.loc[X_train.index, 'close'].max(), alpha=0.2, color='blue', label='Treino')
ax1.fill_between(X_test.index, df_clean.loc[X_test.index, 'close'].min(), 
                df_clean.loc[X_test.index, 'close'].max(), alpha=0.2, color='red', label='Teste')
ax1.set_title('Divisão Treino/Teste - Preços PETR4.SA')
ax1.set_ylabel('Preço (R$)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Target com divisão
ax2.plot(df_clean.index, df_clean['target'], alpha=0.7, color='purple', label='Target')
ax2.axvline(x=split_date, color='red', linestyle='--', alpha=0.8, label='Divisão Treino/Teste')
ax2.fill_between(X_train.index, -0.1, 1.1, alpha=0.2, color='blue', label='Treino')
ax2.fill_between(X_test.index, -0.1, 1.1, alpha=0.2, color='red', label='Teste')
ax2.set_title('Target (Tendência) - Divisão Treino/Teste')
ax2.set_ylabel('Tendência (0=Descida, 1=Subida)')
ax2.set_xlabel('Data')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Treinamento de Múltiplos Modelos

Vamos treinar diferentes algoritmos de machine learning e compará-los.

In [ ]:
# Preparar pipelines com normalização
models = {
    'Logistic Regression': Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', LogisticRegression(random_state=42, max_iter=1000))
    ]),
    
    'Random Forest': Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1))
    ]),
    
    'Gradient Boosting': Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', GradientBoostingClassifier(n_estimators=100, random_state=42))
    ]),
    
    'SVM': Pipeline([
        ('scaler', RobustScaler()),
        ('classifier', SVC(probability=True, random_state=42))
    ])
}

# Treinar e avaliar cada modelo
results = {}
trained_models = {}

print("🚀 Iniciando treinamento dos modelos...\n")

for name, model in models.items():
    print(f"📚 Treinando {name}...")
    
    # Treinamento
    model.fit(X_train, y_train)
    
    # Previsões
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    y_pred_proba_test = model.predict_proba(X_test)[:, 1]
    
    # Métricas
    train_acc = accuracy_score(y_train, y_pred_train)
    test_acc = accuracy_score(y_test, y_pred_test)
    
    # Cross-validation no conjunto de treino
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    
    results[name] = {
        'train_accuracy': train_acc,
        'test_accuracy': test_acc,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'predictions': y_pred_test,
        'probabilities': y_pred_proba_test
    }
    
    trained_models[name] = model
    
    print(f"   ✅ Acurácia Treino: {train_acc:.4f}")
    print(f"   ✅ Acurácia Teste:  {test_acc:.4f}")
    print(f"   ✅ CV (média±std):  {cv_scores.mean():.4f}±{cv_scores.std():.4f}")
    print()

print("🎉 Treinamento concluído!")

In [ ]:
# Criar ensemble (Voting Classifier) com os melhores modelos
print("🤝 Criando modelo ensemble...")

# Selecionar modelos base para o ensemble
ensemble_models = [
    ('rf', trained_models['Random Forest']),
    ('gb', trained_models['Gradient Boosting']),
    ('lr', trained_models['Logistic Regression'])
]

ensemble = VotingClassifier(estimators=ensemble_models, voting='soft')
ensemble.fit(X_train, y_train)

# Avaliar ensemble
y_pred_ensemble_train = ensemble.predict(X_train)
y_pred_ensemble_test = ensemble.predict(X_test)
y_pred_ensemble_proba = ensemble.predict_proba(X_test)[:, 1]

ensemble_train_acc = accuracy_score(y_train, y_pred_ensemble_train)
ensemble_test_acc = accuracy_score(y_test, y_pred_ensemble_test)
ensemble_cv_scores = cross_val_score(ensemble, X_train, y_train, cv=5, scoring='accuracy')

results['Ensemble'] = {
    'train_accuracy': ensemble_train_acc,
    'test_accuracy': ensemble_test_acc,
    'cv_mean': ensemble_cv_scores.mean(),
    'cv_std': ensemble_cv_scores.std(),
    'predictions': y_pred_ensemble_test,
    'probabilities': y_pred_ensemble_proba
}

trained_models['Ensemble'] = ensemble

print(f"✅ Ensemble - Acurácia Treino: {ensemble_train_acc:.4f}")
print(f"✅ Ensemble - Acurácia Teste:  {ensemble_test_acc:.4f}")
print(f"✅ Ensemble - CV (média±std):  {ensemble_cv_scores.mean():.4f}±{ensemble_cv_scores.std():.4f}")

## 7. Avaliação dos Modelos

Vamos comparar o desempenho de todos os modelos e identificar o melhor.

In [ ]:
# Criar DataFrame com resultados
results_df = pd.DataFrame({
    'Modelo': list(results.keys()),
    'Acurácia Treino': [results[model]['train_accuracy'] for model in results.keys()],
    'Acurácia Teste': [results[model]['test_accuracy'] for model in results.keys()],
    'CV Média': [results[model]['cv_mean'] for model in results.keys()],
    'CV Desvio': [results[model]['cv_std'] for model in results.keys()]
})

# Ordenar por acurácia no teste
results_df = results_df.sort_values('Acurácia Teste', ascending=False)

print("📊 RESULTADOS COMPARATIVOS DOS MODELOS")
print("=" * 60)
display(results_df.round(4))

# Identificar o melhor modelo
best_model_name = results_df.iloc[0]['Modelo']
best_accuracy = results_df.iloc[0]['Acurácia Teste']

print(f"\n🏆 MELHOR MODELO: {best_model_name}")
print(f"📈 Acurácia no teste: {best_accuracy:.4f} ({best_accuracy*100:.2f}%)")

# Verificar se atende o critério de 75%
if best_accuracy >= 0.75:
    print(f"✅ OBJETIVO ATINGIDO! Acurácia ≥ 75%")
else:
    print(f"❌ Objetivo não atingido. Acurácia atual: {best_accuracy*100:.2f}% (meta: 75%)")
    print("💡 Sugestões para melhoria:")
    print("   - Adicionar mais features")
    print("   - Otimizar hiperparâmetros")
    print("   - Usar mais dados históricos")
    print("   - Aplicar técnicas de feature selection")

In [ ]:
# Visualização comparativa dos modelos
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# 1. Comparação de acurácias
x_pos = np.arange(len(results_df))
ax1.bar(x_pos - 0.2, results_df['Acurácia Treino'], 0.4, label='Treino', alpha=0.8)
ax1.bar(x_pos + 0.2, results_df['Acurácia Teste'], 0.4, label='Teste', alpha=0.8)
ax1.axhline(y=0.75, color='red', linestyle='--', alpha=0.7, label='Meta 75%')
ax1.set_xlabel('Modelos')
ax1.set_ylabel('Acurácia')
ax1.set_title('Comparação de Acurácias')
ax1.set_xticks(x_pos)
ax1.set_xticklabels(results_df['Modelo'], rotation=45)
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Cross-validation scores
ax2.errorbar(x_pos, results_df['CV Média'], yerr=results_df['CV Desvio'], 
            fmt='o', capsize=5, capthick=2, markersize=8)
ax2.axhline(y=0.75, color='red', linestyle='--', alpha=0.7, label='Meta 75%')
ax2.set_xlabel('Modelos')
ax2.set_ylabel('Acurácia CV')
ax2.set_title('Cross-Validation (5-fold)')
ax2.set_xticks(x_pos)
ax2.set_xticklabels(results_df['Modelo'], rotation=45)
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Matriz de confusão do melhor modelo
best_predictions = results[best_model_name]['predictions']
cm = confusion_matrix(y_test, best_predictions)
sns.heatmap(cm, annot=True, fmt='d', ax=ax3, cmap='Blues',
           xticklabels=['Descida', 'Subida'], yticklabels=['Descida', 'Subida'])
ax3.set_title(f'Matriz de Confusão - {best_model_name}')
ax3.set_xlabel('Predito')
ax3.set_ylabel('Real')

# 4. Distribuição de probabilidades
best_probabilities = results[best_model_name]['probabilities']
ax4.hist(best_probabilities[y_test == 0], bins=20, alpha=0.7, label='Descida Real', color='red')
ax4.hist(best_probabilities[y_test == 1], bins=20, alpha=0.7, label='Subida Real', color='green')
ax4.axvline(x=0.5, color='black', linestyle='--', alpha=0.7, label='Threshold 0.5')
ax4.set_xlabel('Probabilidade Predita')
ax4.set_ylabel('Frequência')
ax4.set_title(f'Distribuição de Probabilidades - {best_model_name}')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Relatório detalhado do melhor modelo
print(f"\n📋 RELATÓRIO DETALHADO - {best_model_name}")
print("=" * 50)
print(classification_report(y_test, best_predictions, target_names=['Descida', 'Subida']))

## 8. Previsões e Visualização dos Resultados

Vamos visualizar as previsões do melhor modelo e analisar os resultados.

In [ ]:
# Preparar dados para visualização
test_dates = X_test.index
test_prices = df_clean.loc[test_dates, 'close']
actual_trends = y_test.values
predicted_trends = results[best_model_name]['predictions']
predicted_probabilities = results[best_model_name]['probabilities']

# Criar DataFrame para análise
analysis_df = pd.DataFrame({
    'Data': test_dates,
    'Preço': test_prices.values,
    'Tendência_Real': actual_trends,
    'Tendência_Predita': predicted_trends,
    'Probabilidade': predicted_probabilities,
    'Acerto': actual_trends == predicted_trends
})

print(f"📊 ANÁLISE DAS PREVISÕES - Últimos 30 dias")
print(f"Período: {test_dates[0].strftime('%Y-%m-%d')} a {test_dates[-1].strftime('%Y-%m-%d')}")
print(f"\n🎯 Resultados:")
print(f"Total de previsões: {len(analysis_df)}")
print(f"Acertos: {analysis_df['Acerto'].sum()} ({analysis_df['Acerto'].mean()*100:.1f}%)")
print(f"Erros: {(~analysis_df['Acerto']).sum()} ({(~analysis_df['Acerto']).mean()*100:.1f}%)")

# Análise por tipo de movimento
subidas_reais = analysis_df[analysis_df['Tendência_Real'] == 1]
descidas_reais = analysis_df[analysis_df['Tendência_Real'] == 0]

print(f"\n📈 Subidas (Real):")
print(f"Total: {len(subidas_reais)}")
print(f"Acertos: {subidas_reais['Acerto'].sum()} ({subidas_reais['Acerto'].mean()*100:.1f}%)")

print(f"\n📉 Descidas (Real):")
print(f"Total: {len(descidas_reais)}")
print(f"Acertos: {descidas_reais['Acerto'].sum()} ({descidas_reais['Acerto'].mean()*100:.1f}%)")

display(analysis_df.head(10))

In [ ]:
# Visualização abrangente dos resultados
fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=(
        'Preços e Previsões', 'Probabilidades de Previsão',
        'Acertos vs Erros', 'Retornos Reais vs Preditos',
        'Distribuição de Acertos', 'Performance por Período'
    ),
    specs=[[{"secondary_y": True}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}]],
    vertical_spacing=0.08
)

# 1. Preços e Previsões
fig.add_trace(
    go.Scatter(x=analysis_df['Data'], y=analysis_df['Preço'], 
              name='Preço PETR4', line=dict(color='blue')),
    row=1, col=1
)

# Adicionar marcadores para previsões
acertos = analysis_df[analysis_df['Acerto']]
erros = analysis_df[~analysis_df['Acerto']]

fig.add_trace(
    go.Scatter(x=acertos['Data'], y=acertos['Preço'], 
              mode='markers', name='Acertos', 
              marker=dict(color='green', size=8, symbol='circle')),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=erros['Data'], y=erros['Preço'], 
              mode='markers', name='Erros', 
              marker=dict(color='red', size=8, symbol='x')),
    row=1, col=1
)

# 2. Probabilidades
fig.add_trace(
    go.Scatter(x=analysis_df['Data'], y=analysis_df['Probabilidade'], 
              name='Prob. Subida', line=dict(color='purple')),
    row=1, col=2
)
fig.add_hline(y=0.5, line_dash="dash", line_color="black", row=1, col=2)

# 3. Acertos vs Erros ao longo do tempo
fig.add_trace(
    go.Scatter(x=analysis_df['Data'], y=analysis_df['Acerto'].astype(int), 
              mode='markers+lines', name='Acertos (1) vs Erros (0)',
              line=dict(color='orange')),
    row=2, col=1
)

# 4. Retornos
returns_real = np.where(analysis_df['Tendência_Real'] == 1, 1, -1)
returns_pred = np.where(analysis_df['Tendência_Predita'] == 1, 1, -1)

fig.add_trace(
    go.Scatter(x=analysis_df['Data'], y=returns_real, 
              name='Retorno Real', line=dict(color='blue')),
    row=2, col=2
)
fig.add_trace(
    go.Scatter(x=analysis_df['Data'], y=returns_pred, 
              name='Retorno Predito', line=dict(color='red', dash='dash')),
    row=2, col=2
)

# 5. Distribuição de acertos
accuracy_by_week = analysis_df.groupby(analysis_df['Data'].dt.week)['Acerto'].mean()
fig.add_trace(
    go.Bar(x=accuracy_by_week.index, y=accuracy_by_week.values, 
          name='Acurácia Semanal'),
    row=3, col=1
)

# 6. Performance acumulada
cumulative_accuracy = analysis_df['Acerto'].expanding().mean()
fig.add_trace(
    go.Scatter(x=analysis_df['Data'], y=cumulative_accuracy, 
              name='Acurácia Acumulada', line=dict(color='green')),
    row=3, col=2
)
fig.add_hline(y=0.75, line_dash="dash", line_color="red", row=3, col=2)

# Configurar layout
fig.update_layout(
    height=1000,
    title_text=f"📊 Análise Completa - {best_model_name} (Acurácia: {best_accuracy:.2%})",
    showlegend=True
)

# Configurar eixos
fig.update_xaxes(title_text="Data", row=3, col=1)
fig.update_xaxes(title_text="Data", row=3, col=2)
fig.update_yaxes(title_text="Preço (R$)", row=1, col=1)
fig.update_yaxes(title_text="Probabilidade", row=1, col=2)
fig.update_yaxes(title_text="Acerto (1) / Erro (0)", row=2, col=1)
fig.update_yaxes(title_text="Retorno", row=2, col=2)
fig.update_yaxes(title_text="Acurácia", row=3, col=1)
fig.update_yaxes(title_text="Acurácia Acumulada", row=3, col=2)

fig.show()

In [ ]:
# Análise de feature importance (para modelos tree-based)
if 'Random Forest' in best_model_name or 'Gradient Boosting' in best_model_name:
    # Extrair feature importances
    if best_model_name == 'Ensemble':
        # Para ensemble, usar Random Forest como referência
        feature_importance = trained_models['Random Forest']['classifier'].feature_importances_
    else:
        feature_importance = trained_models[best_model_name]['classifier'].feature_importances_
    
    # Criar DataFrame de importâncias
    importance_df = pd.DataFrame({
        'Feature': selected_features,
        'Importance': feature_importance
    }).sort_values('Importance', ascending=False)
    
    print("🎯 TOP 10 FEATURES MAIS IMPORTANTES:")
    display(importance_df.head(10))
    
    # Visualizar importâncias
    plt.figure(figsize=(12, 8))
    top_features = importance_df.head(15)
    plt.barh(range(len(top_features)), top_features['Importance'])
    plt.yticks(range(len(top_features)), top_features['Feature'])
    plt.xlabel('Importância')
    plt.title(f'Top 15 Features - {best_model_name}')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

# Resumo final
print("\n" + "="*60)
print("🏆 RESUMO FINAL DO PROJETO")
print("="*60)
print(f"📊 Dados analisados: {len(df_clean)} registros")
print(f"🎯 Período de teste: {len(y_test)} dias (últimos 30 dias)")
print(f"🤖 Melhor modelo: {best_model_name}")
print(f"📈 Acurácia obtida: {best_accuracy:.4f} ({best_accuracy*100:.2f}%)")
print(f"🎯 Meta de acurácia: 75%")

if best_accuracy >= 0.75:
    print(f"✅ SUCESSO: Meta atingida!")
    print(f"💰 O modelo pode ser usado para apoiar decisões de trading")
else:
    print(f"❌ Meta não atingida (diferença: {(0.75 - best_accuracy)*100:.2f}%)")
    print(f"💡 Recomendações para melhoria no futuro")

print(f"\n📝 Features utilizadas: {len(selected_features)}")
print(f"⏰ Período de análise: {df_clean.index[0].strftime('%Y-%m-%d')} a {df_clean.index[-1].strftime('%Y-%m-%d')}")
print("="*60)

## 📋 Conclusões e Próximos Passos

### Principais Resultados
- **Modelo desenvolvido**: Sistema de previsão de tendência para PETR4.SA
- **Dados utilizados**: ~2 anos de dados históricos
- **Features criadas**: Indicadores técnicos, médias móveis, volatilidade, etc.
- **Modelos testados**: Logistic Regression, Random Forest, Gradient Boosting, SVM, Ensemble
- **Período de teste**: Últimos 30 dias de dados

### Metodologia Aplicada
1. ✅ **Coleta de dados**: yfinance para dados históricos da PETR4.SA
2. ✅ **Engenharia de features**: Criação de 20+ indicadores técnicos
3. ✅ **Divisão temporal**: Treino/teste respeitando a natureza temporal
4. ✅ **Múltiplos modelos**: Comparação de diferentes algoritmos
5. ✅ **Ensemble learning**: Combinação dos melhores modelos
6. ✅ **Avaliação rigorosa**: Métricas completas e visualizações

### Próximos Passos (se necessário)
- **Otimização de hiperparâmetros**: GridSearch mais detalhado
- **Mais features**: Dados macroeconômicos, sentimento de mercado
- **Modelos avançados**: LSTM, Transformer para séries temporais
- **Validation**: Walk-forward validation para robustez temporal
- **Risk management**: Incorporar stop-loss e take-profit

### ⚠️ Avisos Importantes
- Este modelo é para fins educacionais e de pesquisa
- Mercados financeiros são imprevisíveis e envolvem riscos
- Sempre consulte profissionais qualificados antes de investir
- Performance passada não garante resultados futuros